In [1]:
import os
import sys
import itertools
import pickle
from glob import glob
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd
from scipy.stats import spearmanr

from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn3
import seaborn as sns

import torch
from torch import nn, optim
from transformers import BertConfig, BertTokenizer, BertModel, BertForMaskedLM#, BertLayer, BertEmbeddings
from transformers.modeling_bert import BertLayer, BertEmbeddings

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# re-load functions
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

In [3]:
DEVICE = 'cpu'

In [4]:
import sys
import pickle
from functools import partial
from glob import glob

import numpy as np
import pandas as pd
import scipy as sp
import torch
from scipy.stats import spearmanr
from tqdm import tqdm

class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """

    def __init__(self):
        self.coef_ = 0

    def _spearmanr_loss(self, coef, X, y, labels):
        """
        Get loss according to
        using current coefficients
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) +
                     [np.inf], labels=labels)

        # return -np.mean(spearmanr(y, X_p).correlation)
        return -spearmanr(y, X_p).correlation

    def fit(self, X, y, initial_coef):
        """
        Optimize rounding thresholds
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        labels = self.labels
        loss_partial = partial(self._spearmanr_loss, X=X, y=y, labels=labels)
        self.coef_ = sp.optimize.minimize(
            # loss_partial, initial_coef, method='Powell')
            loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        labels = self.labels
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) +
                      [np.inf], labels=labels)
        # [np.inf], labels=[0, 1, 2, 3])

    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']

    def set_labels(self, labels):
        self.labels = labels

In [5]:
# sys.path.append('../scripts/')
# from get_optR3 import compute_spearmanr, get_opt_y_pred

import os
import pickle
import sys
from functools import partial
from glob import glob

import numpy as np
import pandas as pd
import scipy as sp
import torch
from scipy.stats import spearmanr
from tqdm import tqdm


class histogramBasedCoefInitializer:
    def __init__(self):
        self.bins = None

    def fit(self, labels):
        self.bins = pd.Series(
            labels).value_counts().sort_index().cumsum().values
        return self

    def predict(self, preds):
        preds = sorted(preds)
        res_threshs = []
        if self.bins is None:
            raise Exception('plz fit at first.')
        for _bin in self.bins[:-1]:
            res_threshs.append((preds[_bin - 1] + preds[_bin]) / 2)
        return res_threshs


class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """

    def __init__(self):
        self.coef_ = 0

    def _spearmanr_loss(self, coef, X, y, labels):
        """
        Get loss according to
        using current coefficients
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) +
                     [np.inf], labels=labels)

        # return -np.mean(spearmanr(y, X_p).correlation)
        return -spearmanr(y, X_p).correlation

    def fit(self, X, y, initial_coef):
        """
        Optimize rounding thresholds
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        labels = self.labels
        loss_partial = partial(self._spearmanr_loss, X=X, y=y, labels=labels)
        self.coef_ = sp.optimize.minimize(
            loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        labels = self.labels
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) +
                      [np.inf], labels=labels)
        # [np.inf], labels=[0, 1, 2, 3])

    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']

    def set_labels(self, labels):
        self.labels = labels


def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        if len(np.unique(col_pred)) == 1:
            if col_pred[0] == np.max(col_trues):
                col_pred[np.argmin(
                    col_pred)] = np.min(col_trues)
            else:
                col_pred[np.argmax(
                    col_pred)] = np.max(col_trues)
        rhos.append(
            spearmanr(
                col_trues,
                col_pred
                #                  + np.random.normal(
                #                     0,
                #                     1e-7,
                #                     col_pred.shape[0])
            ).correlation)
    return rhos


def get_best_ckpt(ckpts):
    ckpt_dicts = []
    for ckpt in ckpts:
        ckpt_dict = {}
        ckpt_dict['ckpt'] = ckpt
        splitted_ckpt = ckpt.split('/')[-1].split('_')
        ckpt_dict['val_metric'] = float(splitted_ckpt[5])
        ckpt_dicts.append(ckpt_dict)
    ckpt_df = pd.DataFrame(ckpt_dicts)
    return ckpt_df.sort_values('val_metric', ascending=False).ckpt.iloc[0]


def get_snapshot_info_df(base_dir):
    res_dicts = []
    for fold in tqdm(list(range(5))):
        fold_ckpts = glob(f'{base_dir}/{fold}/*.pth')
        for ckpt in fold_ckpts:
            res_dict = {}
            splitted_ckpt = ckpt.split('/')[-1].split('_')
            res_dict['ckpt_filename'] = ckpt
            res_dict['fold'] = int(splitted_ckpt[1])
            res_dict['epoch'] = int(splitted_ckpt[3])
            res_dict['val_loss'] = float(splitted_ckpt[4])
            res_dict['val_metric'] = float(splitted_ckpt[5])
            res_dicts.append(res_dict)
    res_df = pd.DataFrame(res_dicts)
    res_df['rank'] = res_df.groupby(['fold']).val_metric.rank()
    return res_df


def get_opt_y_pred(y_true, y_pred, num_labels):
    optRs = []
    opt_y_preds = []

    # for i in tqdm(list(range(21))):
    for i in range(num_labels):
        optR = OptimizedRounder()
        labels = np.sort(np.unique(y_true[:, i]))
        optR.set_labels(labels)
        initer = histogramBasedCoefInitializer().fit(y_true[:, i])
        opt_thresh = initer.predict(y_pred[:, i])
        optR.fit(y_pred[:, i], y_true[:, i], opt_thresh)
        # opt_threshs.append(optR.coefficients())
        # opt_threshs[i] = optR.coefficients()
        optRs.append(optR)
        opt_y_preds.append((optR.predict(y_pred[:, i], optR.coefficients())))

    opt_y_preds = np.asarray(opt_y_preds).T
    return optRs, opt_y_preds


def opt(BASE_PATH, num_labels=30, snapshot_num=2):
    snapshot_df = get_snapshot_info_df(BASE_PATH)

    snapshot_dicts = {}
    state_dict_dicts = {}
    for fold in tqdm(list(range(5))):
        snapshot_dict = {}
        state_dict_dict = {}
        for i, row in snapshot_df.query(f'fold == {fold}').sort_values(
                'rank', ascending=False).reset_index(drop=True).iterrows():
            if i >= snapshot_num:
                continue
            ckpt = torch.load(row['ckpt_filename'])
            state_dict_dict[i] = ckpt['model_state_dict']
            if i == 0:
                qa_ids = ckpt['val_qa_ids']
                qa_ids_argsort = np.argsort(qa_ids)
                snapshot_dict['y_trues'] = [
                    ckpt['val_y_trues'][qa_ids_argsort]]
                snapshot_dict['y_preds'] = [
                    ckpt['val_y_preds'][qa_ids_argsort]]
            else:
                qa_ids = ckpt['val_qa_ids']
                qa_ids_argsort = np.argsort(qa_ids)
                snapshot_dict['y_trues'].append(
                    ckpt['val_y_trues'][qa_ids_argsort])
                snapshot_dict['y_preds'].append(
                    ckpt['val_y_preds'][qa_ids_argsort])
        snapshot_dicts[fold] = snapshot_dict
        state_dict_dicts[fold] = state_dict_dict

    y_preds = np.concatenate(
        [np.average(snapshot_dicts[fold]['y_preds'][:snapshot_num], axis=0)
         for fold in range(5)])
    y_trues = np.concatenate(
        [snapshot_dicts[fold]['y_trues'][0]
         for fold in range(5)])

    reses = []
    optRs = []

    for i in tqdm(list(range(num_labels))):
        y_pred = y_preds[:, i]
        y_true = y_trues[:, i]

        y_pred_argmax = np.argmax(y_pred)
        y_pred_argmin = np.argmin(y_pred)

        optR = OptimizedRounder()
        labels = np.sort(np.unique(y_true))
        optR.set_labels(labels)
        initer = histogramBasedCoefInitializer().fit(y_true)
        initial_coef = initer.predict(y_pred)
        optR.fit(y_pred, y_true, initial_coef=initial_coef)
        optRs.append(optR)
        res = optR.predict(y_pred, optR.coefficients())

        if len(np.unique(res)) == 1:
            if np.unique(res) == res[y_pred_argmax]:
                res[y_pred_argmin] = np.min(y_true)
            else:
                res[y_pred_argmax] = np.max(y_true)

        reses.append(res)
    reses = np.asarray(reses).T

    with open(f'{BASE_PATH}/optRs.pkl', 'wb') as fout:
        pickle.dump(optRs, fout)
    with open(f'{BASE_PATH}/snapshot_dicts.pkl', 'wb') as fout:
        pickle.dump(snapshot_dicts, fout)
    if not os.path.exists(f'{BASE_PATH}/state_dicts'):
        os.mkdir(f'{BASE_PATH}/state_dicts')
    for fold in range(5):
        for rank in range(snapshot_num):
            with open(f'{BASE_PATH}/state_dicts/fold_{fold}_rank_{rank}_state_dict.pkl', 'wb') as fout:
                pickle.dump(state_dict_dicts[fold][rank], fout)
    # with open(f'{BASE_PATH}/state_dict_dicts.pkl', 'wb') as fout:
    #     pickle.dump(state_dict_dicts, fout)

    original_score = compute_spearmanr(y_trues, y_preds)
    print(f'original_score: {original_score}')
    print(f'original_score: {np.mean(original_score)}')

    res_score = compute_spearmanr(y_trues, reses)
    print(f'res_score: {res_score}')
    print(f'res_score_mean: {np.mean(res_score)}')

    return res_score

In [6]:
class histogramBasedCoefInitializer:
    def __init__(self):
        self.bins = None

    def fit(self, labels):
        self.bins = pd.Series(labels).value_counts().sort_index().cumsum().values
        return self

    def predict(self, preds):
        preds = sorted(preds)
        res_threshs = []
        if self.bins is None:
            raise Exception('plz fit at first.')
        for _bin in self.bins[:-1]:
            res_threshs.append((preds[_bin - 1] + preds[_bin]) / 2)
        return res_threshs

## snapshot 済みの model を load

In [41]:
CKPT_DIR = '../mnt/checkpoints'

with open(f'{CKPT_DIR}/e121/snapshot_dicts.pkl', 'rb') as fin:
    bert_question_dict = pickle.load(fin)
with open(f'{CKPT_DIR}/e125/snapshot_dicts.pkl', 'rb') as fin:
    bert_answer_dict = pickle.load(fin)
    
with open(f'{CKPT_DIR}/e126/snapshot_dicts.pkl', 'rb') as fin:
    roberta_question_dict = pickle.load(fin)
with open(f'{CKPT_DIR}/e127/snapshot_dicts.pkl', 'rb') as fin:
    roberta_answer_dict = pickle.load(fin)
    
with open(f'{CKPT_DIR}/e128/snapshot_dicts.pkl', 'rb') as fin:
    xlnet_question_dict = pickle.load(fin)
with open(f'{CKPT_DIR}/e129/snapshot_dicts.pkl', 'rb') as fin:
    xlnet_answer_dict = pickle.load(fin)
    
with open(f'{CKPT_DIR}/e145/snapshot_dicts.pkl', 'rb') as fin:
    lstm_question_dict = pickle.load(fin)
with open(f'{CKPT_DIR}/e147/snapshot_dicts.pkl', 'rb') as fin:
    lstm_answer_dict = pickle.load(fin)

In [42]:
def _get_y_trues_and_y_preds_from_snapshot_dicts(snapshot_dicts, single):
    y_trues, y_preds = [], []
    for fold in range(5):
        if single:
            y_trues.append(snapshot_dicts[fold]['y_trues'][0])
            y_preds.append(snapshot_dicts[fold]['y_preds'][0])
        else:
            y_trues.append(np.average(snapshot_dicts[fold]['y_trues'], axis=0))
            y_preds.append(np.average(snapshot_dicts[fold]['y_preds'], axis=0))
    y_trues = np.concatenate(y_trues)
    y_preds = np.concatenate(y_preds)
    return y_trues, y_preds

def get_y_trues_and_y_preds_from_QA_snapshota_dicts(Q_snapshot_dicts, A_snapshot_dicts, single=False):
    q_y_trues, q_y_preds = _get_y_trues_and_y_preds_from_snapshot_dicts(Q_snapshot_dicts, single)
    a_y_trues, a_y_preds = _get_y_trues_and_y_preds_from_snapshot_dicts(A_snapshot_dicts, single)
    y_trues = np.concatenate([q_y_trues, a_y_trues], axis=1)
    y_preds = np.concatenate([q_y_preds, a_y_preds], axis=1)
    return y_trues, y_preds

In [10]:
# %debug
bert_y_trues, bert_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(bert_question_dict, bert_answer_dict)
roberta_y_trues, roberta_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(roberta_question_dict, roberta_answer_dict)
xlnet_y_trues, xlnet_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(xlnet_question_dict, xlnet_answer_dict)
lstm_y_trues, lstm_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(lstm_question_dict, lstm_answer_dict)

In [11]:
# %debug
single_bert_y_trues, single_bert_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(bert_question_dict, bert_answer_dict, single=True)
single_roberta_y_trues, single_roberta_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(roberta_question_dict, roberta_answer_dict, single=True)
single_xlnet_y_trues, single_xlnet_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(xlnet_question_dict, xlnet_answer_dict, single=True)
single_lstm_y_trues, single_lstm_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(lstm_question_dict, lstm_answer_dict, single=True)

In [12]:
# 良さそう
(bert_y_trues == roberta_y_trues).all(), (bert_y_trues == xlnet_y_trues).all()

(True, True)

In [13]:
(bert_y_preds == roberta_y_preds).all(), (bert_y_preds == xlnet_y_preds).all()

(False, False)

## 混ぜてみる
 - 4 つ全て混ぜてみる
 - 3/4 の　snapshot 混ぜてみる？

In [14]:
def blend_and_evaluate(y_trues, y_preds_list, eval_func, weights=None):
    if weights:
        y_preds = np.average(y_preds_list, axis=0, weights=weights)
    else:
        y_preds = np.average(y_preds_list, axis=0)
    eval_scores = eval_func(y_trues, y_preds)
    optRs, opt_y_preds = get_opt_y_pred(y_trues, y_preds, num_labels=30)
    opt_eval_scores = eval_func(y_trues, opt_y_preds)
    print(f'original_score: {np.mean(eval_scores)}')
    print(f'opt_score: {np.mean(opt_eval_scores)}')
    return eval_scores, opt_eval_scores, optRs

#### 試しに single, top-2 snapshots を見てみる

In [15]:
blend_and_evaluate(single_bert_y_trues, [single_bert_y_preds,  ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [single_roberta_y_preds, ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [single_xlnet_y_preds], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [single_lstm_y_preds], compute_spearmanr)
print('fini!')

original_score: 0.41006427247645066
opt_score: 0.43661262426104713
original_score: 0.40685081230797066
opt_score: 0.4324844364469768
original_score: 0.40631606638648937
opt_score: 0.4325393031770092
original_score: 0.3079114851985878
opt_score: 0.3141410535627739
fini!


In [16]:
blend_and_evaluate(single_bert_y_trues, [bert_y_preds,  ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [roberta_y_preds, ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [xlnet_y_preds], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [lstm_y_preds], compute_spearmanr)
print('fini!')

original_score: 0.41399265956965503
opt_score: 0.4421711714787299
original_score: 0.41044252802107223
opt_score: 0.44857486569826094
original_score: 0.4134497336519045
opt_score: 0.44027891255029494
original_score: 0.31635984791045985
opt_score: 0.3178494225535172
fini!


#### all blends がどうなるかを見てみる

In [25]:
blend_and_evaluate(single_bert_y_trues, [single_bert_y_preds,  single_roberta_y_preds, single_xlnet_y_preds, single_lstm_y_preds], compute_spearmanr,
                                    weights=(0.30, 0.30, 0.30, 0.10))

original_score: 0.4265407213529961
opt_score: 0.4609008774873298


([0.39639992611659136,
  0.6398757854970027,
  0.4181188312517148,
  0.33368880331875894,
  0.37175711445019577,
  0.4442516458433466,
  0.3716371636458366,
  0.5075011507745479,
  0.6183819704360135,
  0.09588100921102036,
  0.4947750329433272,
  0.7604307055998343,
  0.36678064912192815,
  0.19575617511045507,
  0.36270080301637186,
  0.4682678520893574,
  0.7913996270068138,
  0.3747384424347622,
  0.6936336411331152,
  0.06970467524846109,
  0.515429822228163,
  0.2969288177040051,
  0.4456907972895716,
  0.1888728104187696,
  0.20666605129106494,
  0.3755903015413732,
  0.7689759106622713,
  0.2954770144149921,
  0.6993416681604873,
  0.22756744262972867],
 [0.3885614189148279,
  0.6386206703059473,
  0.4840246339863777,
  0.31481972795939245,
  0.37731805791904577,
  0.49257151305258484,
  0.37403725759263273,
  0.5157266778343519,
  0.627386477826103,
  0.1214141114083528,
  0.4772137915410743,
  0.7747954745520901,
  0.5504726215862098,
  0.32667667572208925,
  0.63098706607545

In [16]:
blend_and_evaluate(single_bert_y_trues, [single_bert_y_preds,  single_roberta_y_preds, single_xlnet_y_preds, single_lstm_y_preds], compute_spearmanr)

original_score: 0.42699472473101263
opt_score: 0.45977228979419354


([0.3959018680425978,
  0.6376227422901483,
  0.4196973332280552,
  0.33296020720420777,
  0.3721781039333722,
  0.44276646534091735,
  0.3689865048551,
  0.5073036153994255,
  0.6166207383029141,
  0.09534578978080581,
  0.49660626644176054,
  0.7610618934855841,
  0.36988017509921556,
  0.1980301707546101,
  0.3636810901573147,
  0.4729040558381175,
  0.7920015817511173,
  0.3753380671329925,
  0.6944034501908416,
  0.0697861951660607,
  0.5134527768645245,
  0.2957352584459093,
  0.44937725156438585,
  0.18812845360084238,
  0.20876209203320678,
  0.3800655094328022,
  0.7692358747291859,
  0.29527280572089165,
  0.6991937070457257,
  0.2275416980977498],
 [0.3880235295239206,
  0.6358612698011991,
  0.4853170142743035,
  0.32374716442578927,
  0.3821651057334961,
  0.49152644343965346,
  0.3705191870536341,
  0.5215127151594829,
  0.6229879927872752,
  0.1200851378669567,
  0.4802412588830681,
  0.7638848742877291,
  0.5471587926359825,
  0.32424651169936425,
  0.6277393287630108,


#### 1, 1, 1, 0 で行くことにするので optR を取得

In [36]:
eval_scores, opt_eval_scores, optRs = blend_and_evaluate(bert_y_trues, [single_bert_y_preds,  single_roberta_y_preds, single_xlnet_y_preds, ], compute_spearmanr)

original_score: 0.42699472473101263
opt_score: 0.45977228979419354


In [37]:
!mkdir ../mnt/datasets/sub_e121_e129_bert1_roberta1_xlnet1

In [38]:
with open('../mnt/datasets/sub_e121_e129_bert1_roberta1_xlnet1/optRs.pkl', 'wb') as fout:
    pickle.dump(optRs, fout)

In [39]:
','.join(([str(i) for i in eval_scores]))

'0.3959018680425978,0.6376227422901483,0.4196973332280552,0.33296020720420777,0.3721781039333722,0.44276646534091735,0.3689865048551,0.5073036153994255,0.6166207383029141,0.09534578978080581,0.49660626644176054,0.7610618934855841,0.36988017509921556,0.1980301707546101,0.3636810901573147,0.4729040558381175,0.7920015817511173,0.3753380671329925,0.6944034501908416,0.0697861951660607,0.5134527768645245,0.2957352584459093,0.44937725156438585,0.18812845360084238,0.20876209203320678,0.3800655094328022,0.7692358747291859,0.29527280572089165,0.6991937070457257,0.2275416980977498'

In [40]:
','.join(([str(i) for i in opt_eval_scores]))

'0.3880235295239206,0.6358612698011991,0.4853170142743035,0.32374716442578927,0.3821651057334961,0.49152644343965346,0.3705191870536341,0.5215127151594829,0.6229879927872752,0.1200851378669567,0.4802412588830681,0.7638848742877291,0.5471587926359825,0.32424651169936425,0.6277393287630108,0.6302329257817295,0.7992714875634022,0.35905894862133686,0.691195937346373,0.24000005368949973,0.5123925928624252,0.29466144765092894,0.4494275327505929,0.1938558117355823,0.20675557294496483,0.37903947369022256,0.7624194582385803,0.27702265266176557,0.6861385420951858,0.2266799298583509'

#### weights どうすればよいか見てみる

In [86]:
np.arange(0, 1.1, 0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [ ]:
for i in np.arange(0, 1.1, 0.1):
    for j in np.arange(0, 1.01- i, 0.1):
        weights = [i, j, 1-i-j]
        print(f' ---------- {weights} -----------')
        eval_scores, opt_eval_scores, optRs = blend_and_evaluate(bert_y_trues,
                                                                                                                 [single_bert_y_preds,  roberta_y_preds, single_xlnet_y_preds],
                                                                                                                 compute_spearmanr,
                                                                                                                 weights=weights)

 ---------- [0.0, 0.0, 1.0] -----------
original_score: 0.3906094115371919
opt_score: 0.4208016538440192
 ---------- [0.0, 0.1, 0.9] -----------
original_score: 0.39690428620399504
opt_score: 0.42660201555875216
 ---------- [0.0, 0.2, 0.8] -----------
original_score: 0.40194251860786706
opt_score: 0.4322132022423592
 ---------- [0.0, 0.30000000000000004, 0.7] -----------
original_score: 0.4058057699680523
opt_score: 0.4386623102560342
 ---------- [0.0, 0.4, 0.6] -----------
original_score: 0.40850785092762126
opt_score: 0.44218874682162584
 ---------- [0.0, 0.5, 0.5] -----------
original_score: 0.41002182412980964
opt_score: 0.4462993778180665
 ---------- [0.0, 0.6000000000000001, 0.3999999999999999] -----------
original_score: 0.41026148164290954
opt_score: 0.4472571871083271
 ---------- [0.0, 0.7000000000000001, 0.29999999999999993] -----------
original_score: 0.4092935254165069
opt_score: 0.4447945948297066
 ---------- [0.0, 0.8, 0.19999999999999996] -----------
original_score: 0.40

## まとめ
 - (ちょっと気に入らんけど) bert * 2, roberta * 2, xlnet * 1 の average が良さそう (* 1, * 1, * 1, * 1 よりも)
 - 多分もう一つ別の良いモデルを入れられれば伸びる

In [44]:
from transformers import RobertaModel
model = RobertaModel.from_pretrained('roberta-base')

In [45]:
model.config

{
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "use_bfloat16": false,
  "vocab_size": 50265
}